# RFC 3986 URI Resolution Implementation

This notebook implements the URI resolution algorithm described in [RFC 3986 Section 5.2](https://datatracker.ietf.org/doc/html/rfc3986#section-5.2).

The algorithm transforms a URI reference and a base URI into a target URI.

In [49]:
from urllib.parse import urlparse, urlunparse, quote, unquote
import re

In [50]:
def remove_dot_segments(path):
    """Removes dot segments from a path according to RFC 3986 Section 5.2.4."""
    input_buffer = path
    output = []

    while input_buffer:
        # Rule A: If the input buffer begins with a prefix of "../" or "./",
        # then remove that prefix from the input buffer; otherwise,
        if input_buffer.startswith('../'):
            input_buffer = input_buffer[3:]
        elif input_buffer.startswith('./'):
            input_buffer = input_buffer[2:]
        # Rule B: if the input buffer begins with a prefix of "/./" or "/.",
        # where "." is a complete path segment, then replace that prefix
        # with "/" in the input buffer; otherwise,
        elif input_buffer.startswith('/./'):
            input_buffer = '/' + input_buffer[3:]
        elif input_buffer == '/.':
            input_buffer = '/'
        # Rule C: if the input buffer begins with a prefix of "/../" or "/..",
        # where ".." is a complete path segment, then replace that prefix
        # with "/" in the input buffer and remove the last segment and its
        # preceding "/" (if any) from the output buffer; otherwise,
        elif input_buffer.startswith('/../'):
            input_buffer = '/' + input_buffer[4:]
            if output:
                output.pop()
        elif input_buffer == '/..':
            input_buffer = '/'
            if output:
                output.pop()
        # Rule D: if the input buffer consists only of "." or "..", then remove
        # that from the input buffer; otherwise,
        elif input_buffer == '.' or input_buffer == '..':
            input_buffer = ''
        # Rule E: move the first path segment in the input buffer to the end of
        # the output buffer, including the initial "/" character (if
        # any) and any subsequent characters up to, but not including, the
        # next "/" character or the end of the input buffer.
        else:
            slash_pos = input_buffer.find('/', 1)
            if slash_pos == -1:
                segment = input_buffer
                input_buffer = ''
            else:
                segment = input_buffer[:slash_pos]
                input_buffer = input_buffer[slash_pos:]
            output.append(segment)

    return ''.join(output)


In [51]:

def merge_paths(base_path, relative_path):
    """Merges a base path and a relative path according to RFC 3986 Section 5.2.3."""
    if not base_path:
        return '/' + relative_path

    # Find the last slash in the base path
    last_slash = base_path.rfind('/')
    if last_slash == -1:
        # No slash in base path (e.g., "abc")
        return relative_path
    else:
        # Merge relative path onto the directory containing the last segment
        return base_path[:last_slash + 1] + relative_path


In [52]:

def resolve_uri(base_uri_str, relative_uri_str):
    """Resolves a relative URI against a base URI according to RFC 3986 Section 5.2."""
    base = urlparse(base_uri_str)
    relative = urlparse(relative_uri_str)

    target_scheme = ''
    target_authority = ''
    target_path = ''
    target_params = ''
    target_query = ''
    target_fragment = ''

    # If the relative URI has a scheme and it's different from the base, treat as absolute
    if relative.scheme and relative.scheme != base.scheme:
        target_scheme = relative.scheme
        target_authority = relative.netloc
        target_path = remove_dot_segments(relative.path)
        target_params = relative.params
        target_query = relative.query
    else:
        if relative.netloc: # Authority component is present
            target_authority = relative.netloc
            target_path = remove_dot_segments(relative.path)
            target_params = relative.params
            target_query = relative.query
        else: # Authority component is not present
            if not relative.path:
                if relative.params:
                    # Special case: relative URI is just params (e.g., ';x')
                    # Use base path's directory, but replace params
                    if base.path:
                        last_slash = base.path.rfind('/')
                        if last_slash != -1:
                            target_path = base.path[:last_slash+1]
                        else:
                            target_path = ''
                    else:
                        target_path = ''
                    target_params = relative.params
                    target_query = relative.query
                else:
                    target_path = base.path
                    target_params = base.params
                    if relative.query:
                        target_query = relative.query
                    else:
                        target_query = base.query
                target_authority = base.netloc
            else: # Path is not empty
                if relative.path.startswith('/'):
                    target_path = remove_dot_segments(relative.path)
                    target_params = relative.params
                else:
                    merged_path = merge_paths(base.path, relative.path)
                    target_path = remove_dot_segments(merged_path)
                    target_params = relative.params
                target_query = relative.query
                target_authority = base.netloc
        target_scheme = base.scheme

    target_fragment = relative.fragment

    # Recompose the target URI
    return urlunparse((target_scheme, target_authority, target_path, target_params, target_query, target_fragment))


## RFC 3986 Section 5.4 Examples

Test the implementation using the examples provided in Section 5.4 of the RFC.

In [53]:
base = "http://a/b/c/d;p?q"

# Normal examples
examples = {
    "g:h"           : "g:h",
    "g"             : "http://a/b/c/g",
    "./g"           : "http://a/b/c/g",
    "g/"            : "http://a/b/c/g/",
    "/g"            : "http://a/g",
    "//g"           : "http://g",
    "?y"            : "http://a/b/c/d;p?y",
    "g?y"           : "http://a/b/c/g?y",
    "#s"            : "http://a/b/c/d;p?q#s",
    "g#s"           : "http://a/b/c/g#s",
    "g?y#s"         : "http://a/b/c/g?y#s",
    ";x"            : "http://a/b/c/;x",
    "g;x"           : "http://a/b/c/g;x",
    "g;x?y#s"       : "http://a/b/c/g;x?y#s",
    ""              : "http://a/b/c/d;p?q",
    "."             : "http://a/b/c/",
    "./"            : "http://a/b/c/",
    ".."            : "http://a/b/",
    "../"           : "http://a/b/",
    "../g"          : "http://a/b/g",
    "../.."         : "http://a/",
    "../../"        : "http://a/",
    "../../g"       : "http://a/g"
}

# Abnormal examples
abnormal_examples = {
    "../../../g"    : "http://a/g",
    "../../../../g" : "http://a/g",
    "/./g"          : "http://a/g",
    "/../g"         : "http://a/g",
    "g."            : "http://a/b/c/g.",
    ".g"            : "http://a/b/c/.g",
    "g.."           : "http://a/b/c/g..",
    "..g"           : "http://a/b/c/..g",
    "./../g"        : "http://a/b/g",
    "./g/."         : "http://a/b/c/g/",
    "g/./h"         : "http://a/b/c/g/h",
    "g/../h"        : "http://a/b/c/h",
    "g;x=1/./y"     : "http://a/b/c/g;x=1/y",
    "g;x=1/../y"    : "http://a/b/c/y",
    "g?y/./x"       : "http://a/b/c/g?y/./x", # Path component is opaque
    "g?y/../x"      : "http://a/b/c/g?y/../x", # Path component is opaque
    "g#s/./x"       : "http://a/b/c/g#s/./x",   # Path component is opaque
    "g#s/../x"      : "http://a/b/c/g#s/../x",    # Path component is opaque
    # Strict vs. Non-strict interpretation (we implement strict)
    "http:g"        : "http://a/b/c/g" # Strict: treats "http" as scheme
    # "http:g"        : "http://a/b/c/g" # Non-strict example from RFC (not implemented here)
}

all_examples = {**examples, **abnormal_examples}

print(f"Base URI: {base}\n")

passed = 0
failed = 0

for relative, expected in all_examples.items():
    result = resolve_uri(base, relative)
    if result == expected:
        print(f"  Rel: {relative:<15} -> PASS: {result}")
        passed += 1
    else:
        print(f"* Rel: {relative:<15} -> FAIL: {result} (Expected: {expected})")
        failed += 1

print(f"\nTotal: {len(all_examples)}, Passed: {passed}, Failed: {failed}")


Base URI: http://a/b/c/d;p?q

  Rel: g:h             -> PASS: g:h
  Rel: g               -> PASS: http://a/b/c/g
  Rel: ./g             -> PASS: http://a/b/c/g
  Rel: g/              -> PASS: http://a/b/c/g/
  Rel: /g              -> PASS: http://a/g
  Rel: //g             -> PASS: http://g
  Rel: ?y              -> PASS: http://a/b/c/d;p?y
  Rel: g?y             -> PASS: http://a/b/c/g?y
  Rel: #s              -> PASS: http://a/b/c/d;p?q#s
  Rel: g#s             -> PASS: http://a/b/c/g#s
  Rel: g?y#s           -> PASS: http://a/b/c/g?y#s
  Rel: ;x              -> PASS: http://a/b/c/;x
  Rel: g;x             -> PASS: http://a/b/c/g;x
  Rel: g;x?y#s         -> PASS: http://a/b/c/g;x?y#s
  Rel:                 -> PASS: http://a/b/c/d;p?q
  Rel: .               -> PASS: http://a/b/c/
  Rel: ./              -> PASS: http://a/b/c/
  Rel: ..              -> PASS: http://a/b/
  Rel: ../             -> PASS: http://a/b/
  Rel: ../g            -> PASS: http://a/b/g
  Rel: ../..           -> PAS

In [54]:
base = "https://example.com/api/openapi"
relative = "shared#/components/requestBodies/Foo"
target = resolve_uri(base, relative)
print(target)

https://example.com/api/shared#/components/requestBodies/Foo


In [55]:
base = "https://example.com/api/shared/foo"
relative = "../schemas/foo"
target = resolve_uri(base, relative)
print(target)

https://example.com/api/schemas/foo
